In [2]:
import pandas as pd
import numpy as np

from models import stack as modelStack

In [3]:
base_dir = '../src/projects/'

## Feature coefficients

In [4]:
featureResults = pd.read_csv("../src/projects/ALS-NUPs-variant-females-1MAF/LogisticRegression/globalFeatures.csv")

In [6]:
import os
# Base directory
base_dir = '../src/projects/'

# Project name - replace 'your_project_name' with the actual name of your project
project_name = '/ALS-NUPs-variant-females-1MAF'

# Creating an empty dataframe to store all the values
all_data = pd.DataFrame()

# Looping over all bootstrap directories
for nth_bootstrap in os.listdir(f'{base_dir}/{project_name}/bootstraps/'):
    bootstrap_dir = f'{base_dir}/{project_name}/bootstraps/{nth_bootstrap}/LogisticRegression/'
    file_path = os.path.join(bootstrap_dir, 'averageGlobalExplanations.csv')
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Reading the CSV file into a dataframe, considering the two-level header
        data = pd.read_csv(file_path, header=[0, 1])
        
        # Renaming the index using the first column and dropping the first column
        data.index = data[('Unnamed: 0_level_0', 'Unnamed: 0_level_1')]
        data = data.drop(('Unnamed: 0_level_0', 'Unnamed: 0_level_1'), axis=1)
        
        # Dropping the first row since it's unnecessary
        data = data.drop(index='feature_name')
        
        # Appending the data to the main dataframe
        all_data = pd.concat([all_data, data])


# Creating an explicit aggregation dictionary
agg_dict = {
    ('feature_importances', 'mean'): ['mean', 'median', 'std', 'min', 'max'],
    ('feature_importances', 'std'): ['mean', 'median', 'std', 'min', 'max']
}

# Grouping by the index (feature_name) and applying the aggregation
result = all_data.groupby(level=0).agg(agg_dict)


In [8]:
result.to_csv('nups_logisticRegression_averageGlobalExplanations.csv', index=True)

## Sample accuracies

In [69]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

gene_sets = {
    "ALS-ALSoD-variant-females-1MAF": "ALSoD",
    "ALS-NUPs-variant-females-1MAF": "NUPs"
}

# Loop through models in modelStack dict to get the data
dataframes = []
for model in modelStack:
    model_folder = model.__class__.__name__
    for gene_set, gene_set_abbrev in gene_sets.items():
        csv_path = os.path.join(base_dir, gene_set, model_folder, "sampleResults.csv")
        df = pd.read_csv(csv_path)
        df["gene_set"] = gene_set_abbrev
        df["model"] = model_folder
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

def normalize_id(id_string):
    if "__" in id_string:
        return id_string.split("__")[1]
    return id_string

combined_df["normalized_id"] = combined_df["id"].apply(normalize_id)

# Define color map for consistency
unique_labels = combined_df["label"].unique()
color_map = {0: 'red', 1: 'blue'}

# Create subplots
num_models = len(modelStack)
fig = make_subplots(rows=num_models, cols=1, subplot_titles=[model.__class__.__name__ for model in modelStack], shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.025)

row_num = 1
for model in modelStack:
    df_ALSoD = combined_df[(combined_df["model"] == model.__class__.__name__) & (combined_df["gene_set"] == "ALSoD")]
    df_NUPs = combined_df[(combined_df["model"] == model.__class__.__name__) & (combined_df["gene_set"] == "NUPs")]

    intersecting_ids = set(df_ALSoD["normalized_id"]).intersection(set(df_NUPs["normalized_id"]))

    df_ALSoD_filtered = df_ALSoD[df_ALSoD["normalized_id"].isin(intersecting_ids)]
    df_NUPs_filtered = df_NUPs[df_NUPs["normalized_id"].isin(intersecting_ids)]

    for label in df_ALSoD_filtered["label"].unique():
        fig.add_trace(
            go.Scatter(x=df_ALSoD_filtered[df_ALSoD_filtered["label"] == label]["accuracy_mean"], 
                       y=df_NUPs_filtered[df_NUPs_filtered["label"] == label]["accuracy_mean"], 
                       mode='markers',
                       name=str(label),
                       marker=dict(color=color_map[label]),
                       legendgroup=str(label), 
                       showlegend=(row_num == 1), 
                       hovertext=df_ALSoD_filtered["id"]),
            row=row_num, col=1)
    
    row_num += 1

title_text = f"ALSoD vs. NUPs sample accuracy, all female cases and controls ({len(intersecting_ids)} samples)"
fig.update_layout(
    title=title_text, 
    xaxis_title="ALSoD Accuracy Mean",
    yaxis_title="NUPs Accuracy Mean",
    height=500*num_models,
)

fig.show()

fig.write_html("ALSoD_vs_NUPs_accuracy.html")


## Genotype data split by haplotype

In [195]:
genotypeFile = "../adhoc analysis/Variant_report_NUPs_fixed_2022-03-28.xlsx"
clinicalDataFile = "../adhoc analysis/ACWM.xlsx"
kGenomesFile = "../adhoc analysis/igsr-1000 genomes phase 3 release.tsv"

In [196]:
genotypes = pd.read_excel(genotypeFile, sheet_name="all cases vs all controls", na_values=["'./."]).set_index(['chrom', 'position', 'Gene'])
clinicalData = pd.read_excel(clinicalDataFile).drop_duplicates(subset='ExternalSubjectId').set_index('ExternalSampleId')
kGenomes = pd.read_csv(kGenomesFile, sep="\t").set_index('Sample name')

/Users/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning:

Unknown extension is not supported and will be removed



In [198]:
caseIDsMask, controlIDsMask = [
        clinicalData['Subject Group']
        .isin(labels)
        .dropna()
        for labels in (
            ['ALS Spectrum MND'],
            ['Non-Neurological Control']
        )
    ]

caseIDs = caseIDsMask[caseIDsMask].index.to_numpy()
controlIDs = controlIDsMask[controlIDsMask].index.to_numpy()

controlIDs = np.hstack([controlIDs, kGenomes.index.to_numpy()])

In [199]:
def find_resolved_ids(ids, columns):
    resolved_ids_dict = {}
    for id in ids:
        for column in columns:
            matched_parts = [substr for substr in column.split('__') if len(substr) > 3 and substr in id]
            if matched_parts:
                resolved_ids_dict[id] = column
                break
    return resolved_ids_dict

resolved_case_ids_dict = find_resolved_ids(caseIDs, genotypes.columns)
resolved_control_ids_dict = find_resolved_ids(controlIDs, genotypes.columns)


In [212]:
resolvedGenotypes = genotypes.loc[:, list(resolved_case_ids_dict.values()) + list(resolved_control_ids_dict.values())].dropna()
resolvedGenotypes.index =  resolvedGenotypes.index.set_levels(resolvedGenotypes.index.levels[1].astype(np.int32), level=1)
resolvedGenotypes = resolvedGenotypes.T

In [213]:
ethnicityColValues, labelColValues, sexColValues = {},{},{}
for clinicalID, genotypeID in resolved_case_ids_dict.items():
    labelColValues[genotypeID] = 1
    ethnicityColValues[genotypeID] = clinicalData.loc[clinicalID,'pct_european']
    sexColValues[genotypeID] = clinicalData.loc[clinicalID,'Sex', ]
    
for clinicalID, genotypeID in resolved_control_ids_dict.items():
    if clinicalID in kGenomes.index:
        labelColValues[genotypeID] = 0
        ethnicityColValues[genotypeID] = kGenomes.loc[clinicalID,'Superpopulation code']
        sexColValues[genotypeID] = kGenomes.loc[clinicalID,'Sex']
    else:
        labelColValues[genotypeID] = 1
        ethnicityColValues[genotypeID] = clinicalData.loc[clinicalID,'pct_european']
        sexColValues[genotypeID] = clinicalData.loc[clinicalID,'Sex', ]

In [214]:
resolvedGenotypes.columns.index = resolvedGenotypes.columns.to_flat_index()

# Insert 'label' column at 0th index
resolvedGenotypes.insert(0, 'label', resolvedGenotypes.index.map(labelColValues))

# Insert 'pct_european' column at 1st index (after 'label')
resolvedGenotypes.insert(1, 'pct_european', resolvedGenotypes.index.map(ethnicityColValues))

# Insert 'sex' column at 2nd index (after 'pct_european')
resolvedGenotypes.insert(2, 'sex', resolvedGenotypes.index.map(sexColValues))


In [215]:
resolvedGenotypes.index = resolvedGenotypes.index.to_flat_index()
resolvedGenotypes = resolvedGenotypes.drop(resolvedGenotypes.columns[-1], axis=1)

In [216]:
resolvedGenotypes

chrom                                 label pct_european  ...         X          
position                                                  ... 103083257 154380773
Gene                                                      ...      NXF3       EMD
ALS__CGND-HDA-04091__NEUHF998PCY          1     0.890906  ...      '0/0      '0/0
aals-ALS__CGND-HDA-04089__NEUEU419NMF     1     0.990332  ...      '0/0      '0/0
aals-ALS__CGND-HDA-04087__NEUAN588EBF     1      0.00001  ...      '0/0      '0/0
aals-ALS__CGND-HDA-04086__NEUDH813DE6     1     0.994694  ...      '0/0      '0/0
aals-ALS__CGND-HDA-04085__NEUXZ486GG5     1     0.960508  ...      '0/0      '0/0
...                                     ...          ...  ...       ...       ...
NA21143                                   0          SAS  ...      '0/0      '0/0
NA21112                                   0          SAS  ...      '0/0      '0/0
NA21117                                   0          SAS  ...      '0/0      '0/0
NA21124                                   0          SAS  ...      '0/0      '0/0
NA21129                                   0          SAS  ...      '0/0      '0/0

[5241 rows x 1467 columns]

In [217]:
resolvedGenotypes.to_csv('alsodGenotypesProcessed.csv')

In [218]:
import pandas as pd
from tqdm import tqdm

# Assuming resolvedGenotypes is your input DataFrame

# Extract the metadata columns
metadata = resolvedGenotypes[['label', 'sex', 'pct_european']]

# Dictionaries to store the new columns for each haplotype
haplotype_1_dict = {'label': metadata['label'], 'sex': metadata['sex'], 'pct_european': metadata['pct_european']}
haplotype_2_dict = {'label': metadata['label'], 'sex': metadata['sex'], 'pct_european': metadata['pct_european']}

# Iterate through all columns from the fourth one onwards
for col in tqdm(resolvedGenotypes.columns[3:]):
    # Remove leading apostrophes from the column data
    resolvedGenotypes[col] = resolvedGenotypes[col].str.strip("'")
    haplotype_split = resolvedGenotypes[col].str.split('/', expand=True)

    # Save the haplotypes to the corresponding dictionary
    haplotype_1_dict[', '.join(str(level) for level in col)] = haplotype_split[0]
    haplotype_2_dict[', '.join(str(level) for level in col)] = haplotype_split[1]

# Convert dictionaries to DataFrames
resolvedGenotypes_haplotype_1 = pd.DataFrame(haplotype_1_dict)
resolvedGenotypes_haplotype_2 = pd.DataFrame(haplotype_2_dict)


100%|██████████| 1464/1464 [00:06<00:00, 235.28it/s]


In [219]:
resolvedGenotypes_haplotype_1.to_csv('nupsGenotypes_haplotype_1.csv')
resolvedGenotypes_haplotype_2.to_csv('nupsGenotypes_haplotype_2.csv')

In [220]:
resolvedGenotypes_haplotype_2

,label,sex,pct_european,"1, 28518705, RCC1","1, 28518779, RCC1","1, 28518818, RCC1","1, 28518821, RCC1","1, 28518880, RCC1","1, 28530601, RCC1","1, 28532259, RCC1","1, 28532739, RCC1","1, 28535919, RCC1","1, 28535942, RCC1","1, 28535958, RCC1","1, 28537990, RCC1","1, 28537996, RCC1","1, 53767992, NDC1","1, 53772404, NDC1","1, 53787193, NDC1","1, 53793240, NDC1","1, 53796760, NDC1","1, 53796974, NDC1","1, 53797046, NDC1","1, 53803951, NDC1","1, 53806439, NDC1","1, 53807732, NDC1","1, 53809712, NDC1","1, 53828138, NDC1","1, 53832492, NDC1","1, 53835513, NDC1","1, 53835565, NDC1","1, 53835571, NDC1","1, 53838251, NDC1","1, 186314662, TPR","1, 186317562, TPR","1, 186318567, TPR","1, 186320378, TPR","1, 186323808, TPR","1, 186331572, TPR","1, 186332195, TPR",...,"22, 20126373, RANBP1","22, 20126596, RANBP1","22, 20126597, RANBP1","22, 20127009, RANBP1","22, 41251099, RANGAP1","22, 41252894, RANGAP1","22, 41252910, RANGAP1","22, 41252953, RANGAP1","22, 41254309, RANGAP1","22, 41254334, RANGAP1","22, 41254367, RANGAP1","22, 41254383, RANGAP1","22, 41254406, RANGAP1","22, 41254410, RANGAP1","22, 41254413, RANGAP1","22, 41254422, RANGAP1","22, 41256048, RANGAP1","22, 41256076, RANGAP1","22, 41256253, RANGAP1","22, 41256809, RANGAP1","22, 41258015, RANGAP1","22, 41264747, RANGAP1","22, 41274619, RANGAP1","22, 45171663, NUP50","22, 45176006, NUP50","22, 45176045, NUP50","22, 45178260, NUP50","22, 45178354, NUP50","22, 45178601, NUP50","22, 45178636, NUP50","22, 45178840, NUP50","22, 45178846, NUP50","22, 45178884, NUP50","22, 45183503, NUP50","22, 45183515, NUP50","22, 45184470, NUP50","X, 41341580, DDX3X","X, 41347884, DDX3X","X, 103083257, NXF3","X, 154380773, EMD"
ALS__CGND-HDA-04091__NEUHF998PCY,1,Male,0.890906,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aals-ALS__CGND-HDA-04089__NEUEU419NMF,1,Male,0.990332,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aals-ALS__CGND-HDA-04087__NEUAN588EBF,1,Male,0.00001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aals-ALS__CGND-HDA-04086__NEUDH813DE6,1,Male,0.994694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
aals-ALS__CGND-HDA-04085__NEUXZ486GG5,1,Male,0.960508,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21143,0,female,SAS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
NA21112,0,male,SAS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
NA21117,0,male,SAS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
NA21124,0,male,SAS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
